In [1]:
from bs4 import BeautifulSoup
import urllib.request

from selenium import webdriver
import pandas as pd
import numpy as np

from collections import OrderedDict

import time
from tqdm import tqdm

from nltk.tokenize import word_tokenize

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
vocab = {} 
preprocessed_sentences = []

In [3]:
date = []
cpy_info = []
link_box = []
company = []
topic = []
checkbox = [1,3,4,14,15,19,24,27,63,65,67] # 11,10번은 따로 23소송도 따로

""" 1:가족친화/녹색경영정보 3:기술도입 4:기술성장기업 14:벌금/과징금/행정조치 15:부도/은행거래정지 19:사채/대출원리금미지급 24:손상차손등 27:어음 위·변조 63:해산/파산 65:회계처리기준위반등 67:횡령
11:대표이사변경 29:영업정지/조업중단등 10:담보제공/채무보증 23:소송 37:재해발생 40:제품수거/파기 50:증자/감자 54:차입 """

"""추출 리스트 1,3,4,14,15,19,24,27,63,65,67 """
start = 2010
end = 2020
section = 3



In [4]:
# 회사명 추출
url = "https://kind.krx.co.kr/investwarn/delcompany.do?method=searchDelCompanyMain#viewer"
driver = webdriver.Chrome('C:/Users/YoungMin/Desktop/chromedriver.exe')
driver.get(url)
driver.find_element_by_id('rAll').click() # 전

time.sleep(0.1)    
driver.find_element_by_xpath('//*[@id="currentPageSize"]/option[4]').click() # show 100 case
time.sleep(0.1)

for i in tqdm(range((end - start)//section + 1)):
    
    sty = driver.find_element_by_id("fromDate")
    #time.sleep(0.1)
    sty.clear()
    #time.sleep(0.1)
    sty.send_keys("{fromD}-01-01".format(fromD= start))
    #time.sleep(0.1)

    edy = driver.find_element_by_id("toDate")
    #time.sleep(0.1)
    edy.clear()
    #time.sleep(0.1)
    edy.send_keys("{toD}-12-31".format(toD = start + 2))

    driver.find_element_by_xpath('//*[@id="searchForm"]/section[1]/div/div[3]/a[1]').click()
    time.sleep(2)

    page = driver.find_element_by_xpath('//*[@id="main-contents"]/section[2]/div[2]/em').text # 총 몇 페이지
    page = page.replace(',','')
    page = int(page)
    page_ = page 
    print('{start}-{end_}년도의 해당 공시자료 수는 {page}입니다.'.format(start=start,end_=start+section-1, page=page))
    
    if page <= 1000:
        # 남은 페이지가 100이 넘을 때
        while page_ >=100:
            print('남은 페이지 수는 {num}입니다.'.format(num = page_))
            for i in range(100): # 공시 정보 수집
                company_value = driver.find_element_by_xpath('/html/body/section[2]/section/article/section[1]/table/tbody/tr[{num}]/td[2]/a'.format(num = i + 1)).text
                company.append(company_value)
                topic_value = driver.find_element_by_xpath('/html/body/section[2]/section/article/section[1]/table/tbody/tr[{num}]/td[4]'.format(num = i + 1)).text
                topic.append(topic_value)
                #time.sleep(0.3)
            print('잔여 날짜 RESPONSE 200')

            for i in range(100): # 공시 날짜 수집
                onclick_value = driver.find_element_by_xpath('/html/body/section[2]/section/article/section[1]/table/tbody/tr[{num}]/td[3]'.format(num = i + 1)).text
                #time.sleep(0.3)
                date.append(onclick_value)
            print('잔여 문서 코드 RESPONSE 200')
            time.sleep(0.2)
            driver.find_element_by_xpath('//*[@id="main-contents"]/section[2]/div[1]/a[{num}]'.format(num = page//100 + 4)).click() # 페이지 넘기는 코드
            time.sleep(2)
            page_ = page_ - 100
    else:
        print('공시문서가 1000개가 넘어 오류가 발생합니다!!')

    # 남은 페이지가 100보다 작을 떄     
    """ if page_ < 100: """
    print('남은 페이지 수는 {num}입니다. 수고했어'.format(num = page_))

    for i in range(page_): # 공시 정보 수집
        company_value = driver.find_element_by_xpath('/html/body/section[2]/section/article/section[1]/table/tbody/tr[{num}]/td[2]/a'.format(num = i + 1)).text
        company.append(company_value)
        topic_value = driver.find_element_by_xpath('/html/body/section[2]/section/article/section[1]/table/tbody/tr[{num}]/td[4]'.format(num = i + 1)).text
        topic.append(topic_value)
        #time.sleep(0.3)
    print('잔여 날짜 RESPONSE 200')

    for i in range(page_): # 공시 날짜 수집
        onclick_value = driver.find_element_by_xpath('/html/body/section[2]/section/article/section[1]/table/tbody/tr[{num}]/td[3]'.format(num = i + 1)).text
        #time.sleep(0.3)
        date.append(onclick_value)
    print('잔여 문서 코드 RESPONSE 200')

    start += 3


  0%|                                                                                            | 0/4 [00:00<?, ?it/s]

2010-2012년도의 해당 공시자료 수는 251입니다.
남은 페이지 수는 251입니다.
잔여 날짜 RESPONSE 200
잔여 문서 코드 RESPONSE 200
남은 페이지 수는 151입니다.
잔여 날짜 RESPONSE 200
잔여 문서 코드 RESPONSE 200
남은 페이지 수는 51입니다. 수고했어
잔여 날짜 RESPONSE 200


 25%|█████████████████████                                                               | 1/4 [00:34<01:44, 34.94s/it]

잔여 문서 코드 RESPONSE 200
2013-2015년도의 해당 공시자료 수는 142입니다.
남은 페이지 수는 142입니다.
잔여 날짜 RESPONSE 200
잔여 문서 코드 RESPONSE 200
남은 페이지 수는 42입니다. 수고했어
잔여 날짜 RESPONSE 200


 50%|██████████████████████████████████████████                                          | 2/4 [00:58<00:56, 28.06s/it]

잔여 문서 코드 RESPONSE 200
2016-2018년도의 해당 공시자료 수는 162입니다.
남은 페이지 수는 162입니다.
잔여 날짜 RESPONSE 200
잔여 문서 코드 RESPONSE 200
남은 페이지 수는 62입니다. 수고했어
잔여 날짜 RESPONSE 200


 75%|███████████████████████████████████████████████████████████████                     | 3/4 [01:24<00:27, 27.33s/it]

잔여 문서 코드 RESPONSE 200
2019-2021년도의 해당 공시자료 수는 147입니다.
남은 페이지 수는 147입니다.
잔여 날짜 RESPONSE 200
잔여 문서 코드 RESPONSE 200
남은 페이지 수는 47입니다. 수고했어
잔여 날짜 RESPONSE 200


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [01:47<00:00, 26.91s/it]

잔여 문서 코드 RESPONSE 200


In [5]:
len(date)

702

In [14]:
text_df = pd.DataFrame({'회사명': company,
                        '공시발생일' : date,
                        '폐지사유' : topic})
text_df

,회사명,공시발생일,폐지사유
0,우리스팩1호,2012-12-21,SPAC 상장예비심사청구서 미제출 등
1,중앙건설,2012-12-11,보통주 주가수준미달
2,알에스넷,2012-12-05,"기업의 계속성, 경영의 투명성 또는 기타 코스닥시장의 건전성 등을 종합적으로 고려하..."
3,큐리어스,2012-11-27,"기업의 계속성, 경영의 투명성 또는 기타 코스닥시장의 건전성 등을 종합적으로 고려하..."
4,넷웨이브,2012-11-17,기업의 계속성 및 경영의 투명성 등을 종합적으로 고려하여 상장폐지 기준에 해당한다고 결정
...,...,...,...
697,스페이스솔루션,2019-02-27,상장폐지 신청('19.1.25)
698,원익테라세미콘,2019-02-20,피흡수합병
699,우리은행,2019-02-13,지주회사(최대주주등)의 완전자회사화 등
700,한국4호스팩,2019-01-29,상장예비심사 청구서 미제출로 관리종목 지정 후 1개월 이내 동 사유 미해소


In [15]:
text_df.to_csv("data/crawling.csv")

In [16]:
text_df["폐지사유"].value_counts()

피흡수합병                                                             59
코스닥시장 이전상장                                                        51
해산 사유 발생                                                          48
상장예비심사 청구서 미제출로 관리종목 지정 후 1개월 이내 동 사유 미해소                         41
신청에 의한 상장폐지                                                       40
                                                                  ..
기업의 계속성, 경영의 투명성 또는 기타 코스닥시장의 건전성 등을 종합적으로 고려하여 상장폐지가 필요하다고 결정     1
상장폐지 신청('17.06.22)                                                 1
2반기 연속 자본잠식률 50% 이상                                                1
감사의견 거절(내부회계관리제도상의 취약점, 계속기업가정의 불확실성 및 재고자산 관련 감사범위 제한)            1
사업보고서 법정제출기한('20.3.30限)내 미제출                                       1
Name: 폐지사유, Length: 157, dtype: int64

In [17]:
n = 0
for i in text_df["폐지사유"]:
    if "부도" in i or "파산" in i or "잠식" in i or "거절" in i:
        n += 1
    else :
        continue
print(n)

223
